## Obtener el error de picota, leyendo el pdf de resulatdos

In [2]:
import fitz  # PyMuPDF

doc = fitz.open("../../data/report.pdf")

for page_num, page in enumerate(doc, start=1):
    text = page.get_text()
    print(f"Página {page_num}:")
    print(text)
    print("-" * 80)


Página 1:
Inferential DataSheet
picota
Model Identifier: 6c0d63e8-0ca5-45ac-bc2f-316db77f921a
https://quassar.io/commits/6c0d63e8-0ca5-45ac-bc2f-316db77f921a
Title: main_48_0_1789027358   Release Date: 2025-06-30
Digital Subject: hogar07
Estimation: 
gastoMonetario:productosAlimenticios11
Inference Error: ±2,53 %
Top Contributing Inputs
Input Variable
Weight (%)
ingresosNetos
16,00%
tasaAhorro
15,00%
comidasTotales
15,00%
ipc
9,00%
Estimation: 
gastoMonetario:bebidasNoAlcoholicas12
Inference Error: ±2,11 %
Top Contributing Inputs
Input Variable
Weight (%)
tasaAhorro
16,00%
ingresosNetos
14,00%
ipc
9,00%
comidasTotales
9,00%
Estimation: 
gastoMonetario:bebidasAlcoholicas21
Inference Error: ±,32 %
Top Contributing Inputs
Estimation: gastoMonetario:tabaco22
Inference Error: ±2,57 %
Top Contributing Inputs
Input Variable
Weight (%)
tasaAhorro
14,00%

--------------------------------------------------------------------------------
Página 2:
Input Variable
Weight (%)
tasaAhorro
16,00%
ingres

In [3]:
import fitz
import re
import pandas as pd

# Ruta a tu PDF
ruta_pdf = "../../data/report.pdf"

# Abrir documento
doc = fitz.open(ruta_pdf)

data = []
subject_actual = None

# Recorrer páginas
for page_num, page in enumerate(doc, start=1):
    text = page.get_text()

    # Buscar Digital Subject en la página
    m_subject = re.search(r"Digital Subject:\s*(\w+)", text)
    if m_subject:
        subject_actual = m_subject.group(1)

    # Buscar bloques de Estimation
    bloques = re.findall(
        r"Estimation:\s*(gastoMonetario:[^\n]*)\nInference Error:\s*±([\d,\.]+)\s*%",
        text
    )

    for var, err in bloques:
        # Quitar prefijo "gastoMonetario:"
        var_clean = var.replace("gastoMonetario:", "")
        # Normalizar decimal (puede venir coma o punto)
        err_val = float(err.replace(",", "."))
        data.append({
            "subject": subject_actual,
            "gastoMonetario": var_clean,
            "inference_error_pct": err_val
        })

# Pasar a DataFrame
df_errors = pd.DataFrame(data)

print(df_errors)


     subject           gastoMonetario  inference_error_pct
0    hogar07  productosAlimenticios11                 2.53
1    hogar07   bebidasNoAlcoholicas12                 2.11
2    hogar07     bebidasAlcoholicas21                 0.32
3    hogar07                 tabaco22                 2.57
4    hogar07      articulosDeVestir31                 1.31
..       ...                      ...                  ...
127  hogar05      proteccionSocial124                 0.58
128  hogar05               seguros125                 1.28
129  hogar05  serviciosFinancieros126                 2.57
130  hogar05        otrosServicios127                 0.19
131  hogar05               remesas128                 0.44

[132 rows x 3 columns]


In [4]:
df_errors

,subject,gastoMonetario,inference_error_pct
0,hogar07,productosAlimenticios11,2.53
1,hogar07,bebidasNoAlcoholicas12,2.11
2,hogar07,bebidasAlcoholicas21,0.32
3,hogar07,tabaco22,2.57
4,hogar07,articulosDeVestir31,1.31
...,...,...,...
127,hogar05,proteccionSocial124,0.58
128,hogar05,seguros125,1.28
129,hogar05,serviciosFinancieros126,2.57
130,hogar05,otrosServicios127,0.19


In [11]:
pivot_df = df_errors.pivot_table(
    index="subject",
    columns="gastoMonetario",
    values="inference_error_pct"
)

# Quitamos nombre de las columnas (solo estético)
pivot_df.columns.name = None

# EXTRA: Ordenar columnas por código numérico
# Extraemos el número final de cada columna
col_codes = pivot_df.columns.to_series().apply(
    lambda x: int(''.join(filter(str.isdigit, x[-4:])))
)

# Ordenamos las columnas según esos códigos
ordered_cols = col_codes.sort_values().index.tolist()

# Reordenamos el DataFrame
pivot_df = pivot_df[ordered_cols]


In [13]:
pd.set_option('display.max_columns', None)  # Mostrar todas las columnas
pivot_df

,productosAlimenticios11,bebidasNoAlcoholicas12,bebidasAlcoholicas21,tabaco22,articulosDeVestir31,calzado32,alquileresRealesDeLaVivienda41,mantenimientoDeLaVivienda43,suministroDeAgua44,electricidadGasOtrosCombustibles45,mueblesRevestimientos51,textilesParaElHogar52,grandesElectrodomesticos53,utensiliosDelHogar54,herramientasCasaJardin55,bienesServiciosParaElHogar56,productosFarmaceuticos61,serviciosMedicosAmbulatorios62,serviciosHospitalarios63,compraDeVehiculos71,usoDeVehiculosPersonales72,serviciosDeTransporte73,serviciosPostales81,equiposTelefonoFax82,serviciosTelefonoFax83,audiovisualesTecnologia91,bienesDuraderosDeOcio92,ocioJardineriaYMascotas93,serviciosRecreativosYCulturales94,prensaYPapeleria95,paquetesTuristicos96,educacionInfantilYPrimaria101,educacionSecundariaYPostsecundaria102,educacionSuperior103,educacionNoFormal104,restauracion111,alojamiento112,cuidadosPersonales121,efectosPersonales123,proteccionSocial124,seguros125,serviciosFinancieros126,otrosServicios127,remesas128
subject,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
hogar01,2.76,2.46,0.47,1.46,1.30,2.23,0.85,0.57,0.44,2.77,0.48,0.63,1.45,0.55,0.36,0.66,1.32,0.45,0.13,1.47,2.14,1.41,0.12,1.12,2.54,1.53,0.08,0.66,0.32,0.72,0.86,0.46,0.49,0.71,0.39,2.01,0.92,1.24,0.19,0.35,0.80,0.62,0.07,0.16
hogar05,4.63,4.62,2.25,2.02,1.46,2.94,1.84,1.16,0.72,4.24,1.45,0.69,2.43,0.54,0.73,0.89,1.40,0.56,0.12,2.03,2.40,1.56,0.11,1.34,2.08,0.82,0.08,0.98,1.92,1.78,1.15,1.07,0.82,0.74,0.71,2.27,1.62,3.34,0.53,0.58,1.28,2.57,0.19,0.44
hogar07,2.53,2.11,0.32,2.57,1.31,1.69,1.20,0.59,1.46,1.68,1.04,0.96,1.52,0.27,0.37,0.60,1.93,0.92,0.10,1.62,1.56,1.08,0.19,2.01,3.25,2.21,0.05,0.96,0.59,0.62,2.09,0.70,0.40,0.49,0.37,2.97,1.46,1.26,0.37,0.63,0.83,1.64,0.13,0.61


In [ ]:
# Guardar el DataFrame final en un archivo TSV
df_errors.to_csv("../../datamarts/inference_errors.tsv", sep="\t", index=False)